# Final

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

In [2]:
data=pd.read_csv("cdc_diabetes_health_indicators.csv")

In [3]:
data.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_binary
0,1,1,1,40,1,0,0,0,0,1,...,0,5,18,15,1,0,9,4,3,0
1,0,0,0,25,1,0,0,1,0,0,...,1,3,0,0,0,0,7,6,1,0
2,1,1,1,28,0,0,0,0,1,0,...,1,5,30,30,1,0,9,4,8,0
3,1,0,1,27,0,0,0,1,1,1,...,0,2,0,0,0,0,11,3,6,0
4,1,1,1,24,0,0,0,1,1,1,...,0,2,3,0,0,0,11,5,4,0


In [4]:
features = [
    "Diabetes_binary","HighBP", "HighChol", "HeartDiseaseorAttack", "Stroke",
    "Smoker", "PhysActivity", "DiffWalk",
    "BMI", "MentHlth", "PhysHlth", "GenHlth",
    "Sex", "Age", "HvyAlcoholConsump",
]

In [5]:
df = data[features]

df = df.dropna()

x = df.drop('Diabetes_binary', axis=1)
y = df['Diabetes_binary']

In [6]:
x.head()

,HighBP,HighChol,HeartDiseaseorAttack,Stroke,Smoker,PhysActivity,DiffWalk,BMI,MentHlth,PhysHlth,GenHlth,Sex,Age,HvyAlcoholConsump
0,1,1,0,0,1,0,1,40,18,15,5,0,9,0
1,0,0,0,0,1,1,0,25,0,0,3,0,7,0
2,1,1,0,0,0,0,1,28,30,30,5,0,9,0
3,1,0,0,0,0,1,0,27,0,0,2,0,11,0
4,1,1,0,0,0,1,0,24,3,0,2,0,11,0


In [7]:
x.iloc[10]

HighBP                   0
HighChol                 0
HeartDiseaseorAttack     0
Stroke                   0
Smoker                   1
PhysActivity             1
DiffWalk                 0
BMI                     25
MentHlth                 0
PhysHlth                 0
GenHlth                  3
Sex                      1
Age                     13
HvyAlcoholConsump        0
Name: 10, dtype: int64

In [8]:
y.iloc[10]

np.int64(1)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=42)

In [10]:
xgb_base = XGBClassifier(
    scale_pos_weight=len(y_train[y_train == 0]) / len(y_train[y_train == 1]),
    eval_metric='aucpr',
    use_label_encoder=False,
    objective='binary:logistic',
    verbosity=1
)

# Grid of hyperparameters to try
param_grid = {
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [3, 4, 5],
    "n_estimators": [100, 300],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0]
}

# Setup GridSearch
grid_search = GridSearchCV(
    estimator=xgb_base,
    param_grid=param_grid,
    scoring='f1',  # or 'recall' if recall is your #1 priority
    cv=3,
    verbose=2,
    n_jobs=-1  # Use all CPU cores
)

# Fit search
grid_search.fit(x_train, y_train)


Fitting 3 folds for each of 72 candidates, totalling 216 fits


C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:11:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='aucpr', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 4, 5], 'n_estimators': [100, 300],
                         'subsample': [0.8, 1.0]},
             scoring='f1', verbose=2)

In [11]:

y_probs = grid_search.best_estimator_.predict_proba(x_test)[:, 1]

# Optional: pick threshold that gives best f1/recall
threshold = 0.65
y_pred = (y_probs > threshold).astype(int)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8053715967623253
Confusion Matrix:
 [[54778 10722]
 [ 4090  6514]]
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.84      0.88     65500
           1       0.38      0.61      0.47     10604

    accuracy                           0.81     76104
   macro avg       0.65      0.73      0.67     76104
weighted avg       0.85      0.81      0.82     76104



In [12]:
param_grid = {
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [3, 4, 5],
    "n_estimators": [100, 300],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0]
}
param_grid = {
    'max_depth': [3, 4, 5],
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'colsample_bytree': [0.7, 0.8],
    'eval_metric': ['aucpr']
}

xgb_clf = xgb.XGBClassifier(use_label_encoder=False)
grid = GridSearchCV(xgb_clf, param_grid, scoring='f1', cv=3, verbose=1)
grid.fit(x_train, y_train)

# ✅ Define the best model
model = grid.best_estimator_


Fitting 3 folds for each of 54 candidates, totalling 162 fits


C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:12:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:12:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:12:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\slowd\OneDrive\Desktop\Projects\MLProjects\Diabeties\diabeties\Lib\site-package

In [13]:
import numpy as np
def predict_diabetes(model, HighBP, HighChol, HeartDiseaseorAttack, Stroke, Smoker, PhysActivity, DiffWalk, BMI, MentHlth, PhysHlth, GenHlth, Sex, Age, HvyAlcoholConsump):
    # Match feature order used during training
    new_patient = [HighBP, HighChol, HeartDiseaseorAttack, Stroke, Smoker, PhysActivity, 
                   DiffWalk, BMI, MentHlth, PhysHlth, GenHlth, 
                   Sex, Age, HvyAlcoholConsump
    ]
    
    # Convert to 2D array for prediction
    input_array = np.array(new_patient).reshape(1, -1)
    
    # Make prediction
    prediction = model.predict(input_array)

    # Display result
    if prediction[0] == 1:
        print("⚠️  The patient is likely diabetic.")
    else:
        print("✅  The patient is likely not diabetic.")
    
    return prediction[0]


In [14]:
predict_diabetes(
    model=model,
    HighBP=1,
    HighChol=1,
    HeartDiseaseorAttack=1,
    Stroke=1,
    Smoker=1,
    PhysActivity=1,
    DiffWalk=1,
    BMI=34, 
    MentHlth=0,
    PhysHlth=7,
    GenHlth=4,
    Sex=0,
    Age=9,
    HvyAlcoholConsump=0,
)

⚠️  The patient is likely diabetic.


np.int64(1)

In [15]:
vals=[1,1,1,1,1,1,1,34,0,7,4,0,9,0]

In [16]:
import shap

# Initialize SHAP TreeExplainer
explainer = shap.TreeExplainer(model)

# Choose one sample to explain, for example:
sample_index = 10
sample_input = x_test.iloc[[sample_index]]  # Keep it 2D


In [17]:
sample_input

,HighBP,HighChol,HeartDiseaseorAttack,Stroke,Smoker,PhysActivity,DiffWalk,BMI,MentHlth,PhysHlth,GenHlth,Sex,Age,HvyAlcoholConsump
244217,0,1,0,0,0,1,0,38,0,0,3,1,8,0


In [18]:
shap_values = explainer.shap_values(sample_input)


In [19]:
shap_values

array([[-0.4309479 ,  0.2811314 , -0.04068073, -0.01428051,  0.0040434 ,
        -0.01078929, -0.05131732,  0.6527537 ,  0.03001606,  0.01103784,
         0.34841993,  0.06135189, -0.05293574,  0.02884781]],
      dtype=float32)

In [20]:
sample_input = [vals] # Keep it 2D
shap_values = explainer.shap_values(sample_input)
shap_values

array([[ 0.4434432 ,  0.21272223,  0.24170646,  0.16120987, -0.02344144,
        -0.02974636,  0.15556441,  0.40477532,  0.03053697, -0.00973983,
         0.6026756 , -0.04643119,  0.17074607,  0.03121883]],
      dtype=float32)

In [21]:
shap_values[0][1]

np.float32(0.21272223)

In [22]:
vals=[1,1,1,1,1,1,1,34,0,7,4,0,9,0]
HighBP
HighChol	
HeartDiseaseorAttack	
Stroke	
Smoker	
PhysActivity	
DiffWalk	
BMI	
MentHlth	
PhysHlth	
GenHlth	
Sex	
Age	
HvyAlcoholConsump
feature_names = [
    "HighBP", "HighChol", "HeartDiseaseorAttack", "Stroke", "Smoker", "PhysActivity", 
    "DiffWalk", "BMI", "MentHlth", "PhysHlth", "GenHlth", "Sex", "Age", "HvyAlcoholConsump"]


NameError: name 'HighBP' is not defined

In [23]:
def explain_single_prediction(patient_features, feature_names, explainer, return_raw=False):
    
    # Preprocess input
    shap_values = explainer.shap_values([patient_features])
    class1_values = shap_values[0]

    if return_raw:
        return class1_values

    # Filter + Format
    explanations = []
    for i, sv in enumerate(class1_values):
        feature_name = feature_names[i]
        value = patient_features[i]
        direction = "increased" if sv > 0 else "decreased" if sv < 0 else "had no effect on"
        explanations.append(
            f"**{feature_name}** (value: {value}) {direction} the likelihood of being diabetic by {abs(sv):.3f}"
        )

    return explanations


In [25]:
patient_input = [1,1,1,1,1,1,1,34,0,7,4,0,9,0]
non_diabetic  = [0,0,0,0,1,1,0,25,0,0,3,0,7,0]
feature_names = [
    "HighBP", "HighChol", "HeartDiseaseorAttack", "Stroke", "Smoker", "PhysActivity", 
    "DiffWalk", "BMI", "MentHlth", "PhysHlth", "GenHlth", "Sex", "Age", "HvyAlcoholConsump"]
explainer = shap.TreeExplainer(model)

explain_single_prediction(patient_input, feature_names, explainer)

['**HighBP** (value: 1) increased the likelihood of being diabetic by 0.443',
 '**HighChol** (value: 1) increased the likelihood of being diabetic by 0.213',
 '**HeartDiseaseorAttack** (value: 1) increased the likelihood of being diabetic by 0.242',
 '**Stroke** (value: 1) increased the likelihood of being diabetic by 0.161',
 '**Smoker** (value: 1) decreased the likelihood of being diabetic by 0.023',
 '**PhysActivity** (value: 1) decreased the likelihood of being diabetic by 0.030',
 '**DiffWalk** (value: 1) increased the likelihood of being diabetic by 0.156',
 '**BMI** (value: 34) increased the likelihood of being diabetic by 0.405',
 '**MentHlth** (value: 0) increased the likelihood of being diabetic by 0.031',
 '**PhysHlth** (value: 7) decreased the likelihood of being diabetic by 0.010',
 '**GenHlth** (value: 4) increased the likelihood of being diabetic by 0.603',
 '**Sex** (value: 0) decreased the likelihood of being diabetic by 0.046',
 '**Age** (value: 9) increased the likel

In [27]:
HEALTHY_RANGES = {
    "HighBP": lambda x: x == 0,
    "HighChol": lambda x: x == 0,
    "CholCheck": lambda x: x == 1,
    "BMI": lambda x: x < 25,  # Healthy BMI < 25
    "Smoker": lambda x: x == 0,
    "Stroke": lambda x: x == 0,
    "HeartDiseaseorAttack": lambda x: x == 0,
    "PhysActivity": lambda x: x == 1,
    "HvyAlcoholConsump": lambda x: x == 0,
    "GenHlth": lambda x: x <= 2,  # Excellent/Very Good
    "MentHlth": lambda x: x <= 5,  # Few bad mental health days
    "PhysHlth": lambda x: x <= 5,  # Few bad physical health days
    "DiffWalk": lambda x: x == 0,
    "Sex": lambda x: True,        # Neutral – don't filter
    "Age": lambda x: True         # Age is risk factor – don't filter
}


In [30]:
def explain_single_prediction(patient_features, feature_names, explainer, return_raw=False):
    # Get SHAP values
    shap_values = explainer.shap_values([patient_features])
    class1_values = shap_values[0]

    if return_raw:
        return class1_values

    # Define filtering logic for each feature
    should_include = {
        "HighBP": lambda x: x == 1,
        "HighChol": lambda x: x == 1,
        "HeartDiseaseorAttack": lambda x: x == 1,
        "Stroke": lambda x: x == 1,
        "Smoker": lambda x: x == 1,
        "PhysActivity": lambda x: x == 0,  # no activity
        "DiffWalk": lambda x: x == 1,
        "BMI": lambda x: x >= 25,
        "MentHlth": lambda x: x > 5,
        "PhysHlth": lambda x: x > 5,
        "GenHlth": lambda x: x >= 3,  # good, fair, poor
        "Sex": lambda x: True,        # no filter
        "Age": lambda x: True,        # Age is always checked when SHAP > 0
        "HvyAlcoholConsump": lambda x: x == 1,
    }

    explanations = []
    for i, sv in enumerate(class1_values):
        feature_name = feature_names[i]
        value = patient_features[i]

        # Include Age if SHAP > 0 regardless of value, others only if filter passes
        if (feature_name == "Age" and sv > 0) or (sv > 0 and should_include.get(feature_name, lambda x: True)(value)):
            direction = "increased"
            explanations.append(
                f"**{feature_name}** (value: {value}) {direction} the likelihood of being diabetic by {abs(sv):.3f}"
            )

    return explanations



In [31]:
patient_input = [1,1,1,1,1,1,1,34,0,7,4,0,9,0]
non_diabetic  = [0,0,0,0,1,1,0,25,0,0,3,0,7,0]
feature_names = [
    "HighBP", "HighChol", "HeartDiseaseorAttack", "Stroke", "Smoker", "PhysActivity", 
    "DiffWalk", "BMI", "MentHlth", "PhysHlth", "GenHlth", "Sex", "Age", "HvyAlcoholConsump"]
explainer = shap.TreeExplainer(model)

explain_single_prediction(patient_input, feature_names, explainer)

['**HighBP** (value: 1) increased the likelihood of being diabetic by 0.443',
 '**HighChol** (value: 1) increased the likelihood of being diabetic by 0.213',
 '**HeartDiseaseorAttack** (value: 1) increased the likelihood of being diabetic by 0.242',
 '**Stroke** (value: 1) increased the likelihood of being diabetic by 0.161',
 '**DiffWalk** (value: 1) increased the likelihood of being diabetic by 0.156',
 '**BMI** (value: 34) increased the likelihood of being diabetic by 0.405',
 '**GenHlth** (value: 4) increased the likelihood of being diabetic by 0.603',
 '**Age** (value: 9) increased the likelihood of being diabetic by 0.171']

In [ ]:
['**HighBP** (value: 1) increased the likelihood of being diabetic by 0.443',
 '**HighChol** (value: 1) increased the likelihood of being diabetic by 0.213',
 '**HeartDiseaseorAttack** (value: 1) increased the likelihood of being diabetic by 0.242',
 '**Stroke** (value: 1) increased the likelihood of being diabetic by 0.161',
 '**Smoker** (value: 1) decreased the likelihood of being diabetic by 0.023',
 '**PhysActivity** (value: 1) decreased the likelihood of being diabetic by 0.030',
 '**DiffWalk** (value: 1) increased the likelihood of being diabetic by 0.156',
 '**BMI** (value: 34) increased the likelihood of being diabetic by 0.405',
 '**MentHlth** (value: 0) increased the likelihood of being diabetic by 0.031',
 '**PhysHlth** (value: 7) decreased the likelihood of being diabetic by 0.010',
 '**GenHlth** (value: 4) increased the likelihood of being diabetic by 0.603',
 '**Sex** (value: 0) decreased the likelihood of being diabetic by 0.046',
 '**Age** (value: 9) increased the likelihood of being diabetic by 0.171',
 '**HvyAlcoholConsump** (value: 0) increased the likelihood of being diabetic by 0.031']

In [32]:
import joblib

# Save model
joblib.dump(model, 'xgb_model.pkl')

# Optional: Save SHAP explainer if you're using TreeExplainer
explainer = shap.TreeExplainer(model)
joblib.dump(explainer, 'shap_explainer.pkl')


['shap_explainer.pkl']